In [ ]:
#中文句子关系
#输入两句话判断是否一句话是下一句的另一句话

In [23]:
#中文分类
# 导入工具包
import os
import sys
import random
import torch.nn as nn
# 添加环境变量
os.environ['TRANSFORMERS_OFFLINE'] = '0'
import torch
from datasets import load_dataset,Dataset
from transformers import BertTokenizer, BertModel
from torch.optim import AdamW
import time
from torch.utils.data import DataLoader
from tqdm import tqdm
#加载分词器
my_tokenizer = BertTokenizer.from_pretrained(r'D:\bert')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
# 2 加载模型
my_model = BertModel.from_pretrained(r'D:\bert').to(device)#和自动模型区别只在于导包不相同

cuda


Some weights of the model checkpoint at D:\bert were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [24]:
#自定义dataset
class MyDataset(Dataset):
    def __init__(self,data_path):
        my_dataset = load_dataset('csv',data_files = data_path,split = 'train')
        #只获取长度大于44
        self.dataset = my_dataset.filter(lambda x:len(x['text'])>44)
        self.len = len(self.dataset)
    def __len__(self):
        
        return self.len
    #根据索引去样本
    def __getitem__(self,idx):
        label = 1#有关系
        text = self.dataset[idx]['text']
        sentence1 = text[0:22]
        sentence2 = text[22:44]
        #产生负样本
        if random.randint(0,1) == 0:
            #重新选择索引
            j = random.randint(0,self.len-1)
            sentence2 = self.dataset[j]['text'][22:44]
            label = 0
        return sentence1,sentence2,label
mydataset = MyDataset('./train.csv')
print(mydataset.dataset)
print(mydataset[0])
print(mydataset[1])
print(mydataset[2])

Dataset({
    features: ['label', 'text'],
    num_rows: 7472
})
('选择珠江花园的原因就是方便，有电动扶梯直接到', '达海边，周围餐馆、食廊、商场、超市、摊位一应', 1)
('15.4寸笔记本的键盘确实爽，基本跟台式机差', '错,温度不高。较安静。装vista性能。4.', 0)
('1.接电源没有几分钟,电源适配器热的不行. ', '2.摄像头用不起来. 3.机盖的钢琴漆，手不', 1)


In [32]:
def collate_fn(data):
    #取出样本
    sents = [i[:2] for i in data]
    labels = [i[2] for i in data]
    #张量处理
    inputs = my_tokenizer.batch_encode_plus(sents,padding = 'max_length',truncation=True,max_length = 50,return_tensors='pt')
    input_ids = inputs['input_ids']
    token_type_ids = inputs['token_type_ids']
    attention_mask = inputs['attention_mask']
    labels = torch.tensor(labels,dtype=torch.long)
    return input_ids, attention_mask, token_type_ids, labels

In [33]:
def test_dataloader():
    #实例化dataset对象
    train_dataset = MyDataset(r'./train.csv')
    #print(train_dataset)
    #print(new_train_dataset)
    #将上述数据进行封装
    train_dataloader = DataLoader(dataset = train_dataset,batch_size = 8,shuffle=True,collate_fn =collate_fn,drop_last=True)
    #遍历
    #for input_ids,token_type_ids,attention_mask,labels in train_dataloader:
        #print('test')
       # print(labels.shape)
        #brea
    return train_dataloader

In [34]:
class Mymodel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Linear(768,2)
    def forward(self,input_ids,token_type_ids,attention_mask):
        #预训练模型的参数不更新
        with torch.no_grad():
            output = my_model(input_ids = input_ids,
                           attention_mask = attention_mask,
                           token_type_ids = token_type_ids)
        #将bert编码后的结果送入输出层
        result = self.fc(output['pooler_output'])
        return result

In [35]:
def train_model():
    #实例化数据源对象
    train_dataset = MyDataset(r'./train.csv')
    #将上述的数据进行封装
    train_dataloader = DataLoader(dataset =train_dataset,batch_size = 8,collate_fn = collate_fn,drop_last = True,shuffle=True)
    #实例化模型
    mymodel = Mymodel().to(device)
    #实例化优化器
    my_optimizer = AdamW(mymodel.parameters(),lr = 5e-4)
    #实例化损失函数
    my_criterion = torch.nn.CrossEntropyLoss()
    # 不训练预训练模型 只让预训练模型计算数据特征 不需要计算梯度
    for param in my_model.parameters():
        param.requires_grad_(False)#删除的时候最原始的模型会进行更新
    #设置训练参数
    epochs = 3
    #设置训练模式
    mymodel.train()
    #开始训练
    for epoch in range(epochs):
        start_time = time.time()
        #开启内部数据迭代
        for idx,(input_ids,token_type_ids,attention_mask,labels_y) in enumerate(tqdm(train_dataloader),start = 1):
            #数据送入模型
            input_ids = input_ids.to(device)
            token_type_ids = token_type_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels_y = labels_y.to(device)
            output = mymodel(input_ids,token_type_ids,attention_mask)
            #print(output)
            #计算损失
            myloss = my_criterion(output,labels_y)
            #print(myloss)
            #梯度清0
            my_optimizer.zero_grad()
            #反向传播
            myloss.backward()
            #梯度更新
            my_optimizer.step()
            if idx % 20 == 0:
                out = output.argmax(dim=1) # [8,2] --> (8,)
                accuracy = (out == labels_y).sum().item() / len(labels_y)
                print(f'轮次:{epoch+1} 迭代数:{idx} 损失:{myloss.item():.6f} 准确率:{accuracy:.3f} 时间:{time.time()-start_time}s')

        # 每个轮次保存模型
        torch.save(mymodel.state_dict(), './nsp_%d.bin' % (epoch + 1))

train_model()

  3%|▎         | 26/934 [00:04<00:52, 17.39it/s]

轮次:1 迭代数:20 损失:0.931127 准确率:0.375 时间:3.9776525497436523s


  5%|▍         | 44/934 [00:04<00:32, 27.03it/s]

轮次:1 迭代数:40 损失:0.893524 准确率:0.375 时间:4.64610481262207s


  7%|▋         | 64/934 [00:05<00:29, 29.85it/s]

...

 97%|█████████▋| 906/934 [00:30<00:00, 30.10it/s]

轮次:3 迭代数:900 损失:0.644459 准确率:0.625 时间:30.606310606002808s


 99%|█████████▉| 925/934 [00:31<00:00, 29.22it/s]

轮次:3 迭代数:920 损失:0.656629 准确率:0.625 时间:31.27694010734558s


100%|██████████| 934/934 [00:31<00:00, 29.43it/s]


In [38]:
def test_model():
    #实例化数据源对象
    test_dataset = MyDataset(r'./test.csv')
    #将上述的数据进行封装
    test_dataloader = DataLoader(dataset =test_dataset,batch_size = 8,collate_fn = collate_fn,drop_last = True,shuffle=True)
    #加载模型
    my_model = Mymodel().to(device)
    my_model.load_state_dict(torch.load('./my_model_1.bin'))
    #定义测试超参数
    total = 0
    acc_num = 0
    my_model.eval()
    #开始测试
    for idx,(input_ids,token_type_ids,attention_mask,labels_y) in enumerate(tqdm(test_dataloader),start = 1):
        input_ids = input_ids.to(device)
        token_type_ids = token_type_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels_y = labels_y.to(device)
        with torch.no_grad():
            output = my_model(input_ids,token_type_ids,attention_mask)
        #计算正确的样本个数
        out = output.argmax(dim=1) # [8,2] --> (8,)
        acc_num = acc_num + (out == labels_y).sum().item() 
        total = total+len(labels_y)
        if idx %5 ==0:
            print(acc_num/total,end =' ')
            print(my_tokenizer.decode(input_ids[0],skip_special_tokens=True),end=' ')
            print('预测值 真实值:',out[0].item(),labels_y[0].item())
test_model()

  8%|▊         | 9/116 [00:00<00:04, 26.38it/s]

0.45 书 的 质 量 很 好 ， 纸 张 很 厚 实 ， 画 面 很 漂 亮 ， 图 画 很 大 ， 但 都 很 幼 稚 ， 不 适 合 三 年 级 的 孩 子 看 ， （ 我 女 预测值 真实值: 0 1
0.5125 1. 有 急 事 出 去 ， 要 们 童 叫 出 租 车 ， 他 们 就 叫 酒 店 里 的 黑 车 ， 价 格 是 普 通 出 租 价 的 两 倍 。 你 提 出 不 要 预测值 真实值: 0 1


 16%|█▌        | 18/116 [00:00<00:03, 27.03it/s]

0.5 有 了 第 一 本 书 的 铺 垫 ， 读 第 二 本 的 时 候 开 始 进 入 状 态 。 基 本 上 第 二 本 就 围 绕 主 角 们 的 能 力 训 练 展 开 ， 预测值 真实值: 1 1
0.4875 突 然 想 起 很 多 年 前 ， 那 时 候 还 在 读 小 学 来 着 ， 记 得 有 次 参 加 作 文 比 赛 ， 然 后 题 目 就 是 《 父 与 子 》 里 的 预测值 真实值: 0 1


 26%|██▌       | 30/116 [00:01<00:03, 27.10it/s]

...

0.5078947368421053 地 理 位 置 不 错 ， 在 市 中 心 繁 华 地 段 ， 去 天 津 街 、 清 一 日 本 客 人 一 起 住 的 对 方 很 中 意 因 为 竟 然 有 预测值 真实值: 0 0
0.5125 我 前 几 天 发 表 了 我 的 看 法 ， 竟 然 被 当 当 工 作 人 员 删 于 航 班 原 因 早 晨 3 点 57 分 到 的 酒 店 ， 我 订 的 是 8 预测值 真实值: 0 0


 93%|█████████▎| 108/116 [00:03<00:00, 27.35it/s]

0.5119047619047619 我 在 京 东 网 买 的 华 硕 笔 记 本 ， 第 二 天 就 发 现 严 重 的 日 离 开 时 ， 大 厅 有 钢 琴 小 提 琴 的 表 演 。 订 房 时 （ 大 预测值 真实值: 0 0
0.5068181818181818 可 能 是 别 人 的 评 论 给 我 的 误 导, 这 本 书 还 是 没 多 大 屏 ， 音 响 效 果 相 当 棒 。 性 价 比 相 当 不 错 ， 500 预测值 真实值: 1 0


100%|██████████| 116/116 [00:04<00:00, 27.07it/s]

0.5054347826086957 酒 店 的 地 理 位 置 不 好 ， 出 门 要 门 童 叫 个 出 租 车 ， 等 了 10 分 钟 门 童 才 把 车 叫 过 来 ， 慢 ！ 洗 澡 花 洒 的 出 预测值 真实值: 0 1
